### EOLT Dataset management

From the full EOLT dataset we clean two different datasets:

    -The RW calibration dataset
    
    -The TR test dataset
    
Those dataset should have the following characteristics:

    -Only one row per SN. (full SN required)
    
    -Keep the last "pass" test of each drawer.
    
    -Not null data.

In [1]:
#Import:
import numpy as np
import pandas as pd

In [2]:
full_df = pd.read_csv('../Data/EOLT_Report.csv')

In [3]:
full_df.head()

,ID,SN,TEST,DAY,@TIME,TESTVERSION,STATUS,FIRST,LAST,SHIFT,...,RESULTNUM8,X8,RESULTNUM9,X9,RESULTNUM10,X10,RESULTNUM11,X11,RESULTNUM12,X12
0,2,DRW521500731,DrawerNwakeSwitch,20160223,154930,1,F,N,Y,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3,DRW521500731,DrawerNwakeSwitch,20160223,154930,1,F,N,Y,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4,DRW,PaperJamSwitch,20170921,165700,1,P,Y,Y,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5,DRW,MediaSensorRoll1,20170921,165905,1,F,Y,Y,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,6,DRW,PaperJamSwitch,20170921,171632,1,P,Y,Y,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
full_df.columns

Index(['ID', 'SN', 'TEST', 'DAY', '@TIME', 'TESTVERSION', 'STATUS', 'FIRST',
       'LAST', 'SHIFT', 'FAMILY', 'STATION', '@TABLE', 'RESULTNUM1', 'X1',
       'RESULTNUM2', 'X2', 'RESULTNUM3', 'X3', 'RESULTNUM4', 'X4',
       'RESULTNUM5', 'X5', 'RESULTNUM6', 'X6', 'RESULTNUM7', 'X7',
       'RESULTNUM8', 'X8', 'RESULTNUM9', 'X9', 'RESULTNUM10', 'X10',
       'RESULTNUM11', 'X11', 'RESULTNUM12', 'X12'],
      dtype='object')

### Cleaning dataset

Columns ID, TESTVERSION, FIRST, LAST, SHIFT, FAMILY, FAMILY, @TABLE are not relevant for the analysis.

In [5]:
#We remove not useful columns:
drop_col = ['ID', 'TESTVERSION', 'FIRST', 'LAST', 'SHIFT', 'FAMILY', 'STATION', '@TABLE']
full_df.drop(drop_col, axis=1, inplace=True)

In [6]:
#SN has a format that is DRWXXXXXXXXX. We remove rows with other SN:

full_df = full_df[full_df.SN != 'DRW']

In [7]:
full_df.shape

(64790, 29)

In [8]:
full_df.head()

,SN,TEST,DAY,@TIME,STATUS,RESULTNUM1,X1,RESULTNUM2,X2,RESULTNUM3,...,RESULTNUM8,X8,RESULTNUM9,X9,RESULTNUM10,X10,RESULTNUM11,X11,RESULTNUM12,X12
0,DRW521500731,DrawerNwakeSwitch,20160223,154930,F,NaN,F,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,DRW521500731,DrawerNwakeSwitch,20160223,154930,F,NaN,F,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16,DRW451501111,PaperJamSwitch,20170922,84340,P,PaperJameSwitchLeft,P,PaperJameSwitchRight,P,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17,DRW451501111,MediaSensor,20170922,84420,P,MediaSensorRoll1,P,MediaSensorRoll2,P,MediaSensorEOD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18,DRW451501111,UserButtonRoll,20170922,84430,P,UserButtonRoll1,P,UserButtonRoll2,P,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
full_df.TEST.unique()

array(['DrawerNwakeSwitch', 'PaperJamSwitch', 'MediaSensor',
       'UserButtonRoll', 'CutterEOT', 'LedsBlink', 'RWCalib', 'TrPWM',
       'CutPWM', 'NIDRCutTime', 'HPSerialNumber', 'CutterLimitSwitchLeft',
       'MediaSensorEOD', 'PaperJamSwitchLeft', 'DrawerNwake',
       'MediaSensorRoll1', 'UserButton', 'PaperJamSwitchRight',
       'UserButtonRoll2', 'CutterLimitSwitchRight', 'MediaSensorRoll2'],
      dtype=object)

We must unify the DAY and TIME column into a datetime column to filter each SN by the last test:

In [10]:
#We create a datetime column that concatenate both day and time columns into only one:
#The column time we ensure to have 6 elements adding a 0 in case it didn't.
convert_time = lambda x: str(x) if len(str(x))==6 else ('0' + str(x))

full_df['datetime'] = full_df['DAY'].map(str) + full_df['@TIME'].apply(convert_time)
full_df['datetime'] = pd.to_numeric(full_df['datetime'])

### Spliting the dataset into the two tests datasets:

#### 1.- RW Calibration:

In [11]:
RWcal = full_df[full_df.TEST == 'RWCalib']

The test columns are divided in two: test name and test result. We delete the first ones and rename the second ones.

In [12]:
#We drop the test name columns:
test_cols = ['RESULTNUM1','RESULTNUM2','RESULTNUM3','RESULTNUM4','RESULTNUM5','RESULTNUM6',
             'RESULTNUM7','RESULTNUM8','RESULTNUM9','RESULTNUM10','RESULTNUM11','RESULTNUM12']
RWcal.drop(test_cols, axis=1, inplace=True)

#We rename all the columns:

RW_columns = ['SN','TEST','day','time','status','A_Indep_Front','A_RollSpeed_Front','A_R2R2W_Front',
             'A_R2R2W_RollSpeed_Front','KM_Front','KM_DEV_Front','A_Indep_Rear','A_RollSpeed_Rear',
             'A_R2R2W_Rear','A_R2R2W_RollSpeed_Rear','KM_Rear','KM_DEV_Rear', 'datetime']

RWcal.columns = RW_columns

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [13]:
#We reorder the columns:

RWcal = RWcal[['SN','TEST','day','time','datetime','status','A_Indep_Front','A_RollSpeed_Front','A_R2R2W_Front',
             'A_R2R2W_RollSpeed_Front','KM_Front','KM_DEV_Front','A_Indep_Rear','A_RollSpeed_Rear',
             'A_R2R2W_Rear','A_R2R2W_RollSpeed_Rear','KM_Rear','KM_DEV_Rear']]

In [14]:
RWcal.head()

,SN,TEST,day,time,datetime,status,A_Indep_Front,A_RollSpeed_Front,A_R2R2W_Front,A_R2R2W_RollSpeed_Front,KM_Front,KM_DEV_Front,A_Indep_Rear,A_RollSpeed_Rear,A_R2R2W_Rear,A_R2R2W_RollSpeed_Rear,KM_Rear,KM_DEV_Rear
34,DRW451501111,RWCalib,20170921,172158,20170921172158,P,0.317183,0.000797,92.715429,0.0,0.041446,0.068585,0.312694,0.000741,71.234738,0.0,0.041134,0.015569
47,DRW381700111,RWCalib,20170922,122641,20170922122641,P,0.095211,0.000954,127.069153,0.0,0.041651,0.034946,0.228156,0.000864,115.710218,0.0,0.041463,0.093739
71,DRW381700101,RWCalib,20170925,133505,20170925133505,P,0.323184,0.001028,101.220874,0.0,0.041169,0.080830,0.357718,0.000806,73.678820,0.0,0.040928,0.047217
72,DRW381700101,RWCalib,20170925,133433,20170925133433,P,0.335312,0.001099,83.877375,0.0,0.041213,0.099533,0.321589,0.000952,68.793419,0.0,0.040929,0.050038
85,DRW381700121,RWCalib,20170925,170409,20170925170409,P,0.359482,0.000864,97.995323,0.0,0.041349,0.048904,0.376642,0.001063,132.648396,0.0,0.041491,0.082657


In [15]:
#Groupby SN and select last PASS
#https://stackoverflow.com/questions/41525911/group-by-pandas-dataframe-and-select-latest-in-each-group

RW_Pass = RWcal[RWcal.status == 'P'].sort_values('datetime').groupby('SN').tail(1)

#We drop the columns TEST and status because they are no longer useful:
RW_Pass.drop(['TEST', 'status'], axis=1, inplace=True)
RW_Pass.head()

,SN,day,time,datetime,A_Indep_Front,A_RollSpeed_Front,A_R2R2W_Front,A_R2R2W_RollSpeed_Front,KM_Front,KM_DEV_Front,A_Indep_Rear,A_RollSpeed_Rear,A_R2R2W_Rear,A_R2R2W_RollSpeed_Rear,KM_Rear,KM_DEV_Rear
10988,DRW091810461,20180302,1711,2018030201711,0.390513,0.000601,128.379043,0.0,0.041950,0.037106,0.468861,0.000664,89.051406,0.0,0.042561,0.053677
11000,DRW091810471,20180302,3725,2018030203725,0.348259,0.000509,129.046143,0.0,0.041847,0.052308,0.338558,0.000449,86.579810,0.0,0.041813,0.041948
552,DRW391710071,20170922,105735,20170922105735,0.407983,0.000620,53.861514,0.0,0.041439,0.031839,0.447917,0.001068,72.697784,0.0,0.041715,0.033923
101,DRW131300011,20170926,91415,20170926091415,0.101011,0.000571,56.738101,0.0,0.041159,0.018542,0.155693,0.000409,65.160779,0.0,0.040837,0.023889
130,DRW381700121,20170926,101728,20170926101728,0.372220,0.000876,115.867352,0.0,0.041370,0.034078,0.400256,0.001093,94.390418,0.0,0.041121,0.084587


In [16]:
RW_Pass.to_csv('../Data/RW.csv', index=False)

#### 2- TR test:

In [17]:
TRtest = full_df[full_df.TEST == 'TrPWM']

In [18]:
TRtest.head()

,SN,TEST,DAY,@TIME,STATUS,RESULTNUM1,X1,RESULTNUM2,X2,RESULTNUM3,...,X8,RESULTNUM9,X9,RESULTNUM10,X10,RESULTNUM11,X11,RESULTNUM12,X12,datetime
48,DRW381700111,TrPWM,20170922,123028,P,TR_PWM_SLOP_FRONT,1.351066,TR_PWM_OFFSET_FRONT,-38.202471,R2_Front,...,9.177749,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20170922123028
73,DRW381700101,TrPWM,20170925,133859,P,TR_PWM_SLOP_FRONT,1.346572,TR_PWM_OFFSET_FRONT,-35.734340,R2_Front,...,9.175934,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20170925133859
90,DRW381700121,TrPWM,20170925,170807,P,TR_PWM_SLOP_FRONT,1.335216,TR_PWM_OFFSET_FRONT,-35.803788,R2_Front,...,9.176898,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20170925170807
103,DRW131300011,TrPWM,20170926,92330,P,TR_PWM_SLOP_FRONT,1.329452,TR_PWM_OFFSET_FRONT,-35.662015,R2_Front,...,9.180915,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20170926092330
120,DRW381700121,TrPWM,20170926,100237,P,TR_PWM_SLOP_FRONT,1.323909,TR_PWM_OFFSET_FRONT,-36.640903,R2_Front,...,9.178672,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20170926100237


In [19]:
#We drop the test name columns:
test_cols = ['RESULTNUM1','RESULTNUM2','RESULTNUM3','RESULTNUM4','RESULTNUM5','RESULTNUM6',
             'RESULTNUM7','RESULTNUM8','RESULTNUM9','RESULTNUM10','RESULTNUM11','RESULTNUM12',
            'X9', 'X10', 'X11', 'X12']
TRtest.drop(test_cols, axis=1, inplace=True)

#We rename all the columns:

TR_columns = ['SN','TEST','day','time','status','TR_PWM_SLOP_FRONT','TR_PWM_OFFSET_FRONT',
              'R2_Front','TR_AccuENC_RATIO_FRONT','TR_PWM_SLOP_REAR','TR_PWM_OFFSET_REAR',
              'R2_Rear','TR_AccuENC_RATIO_REAR','datetime']

TRtest.columns = TR_columns

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [20]:
#We reorder the columns:

TRtest = TRtest[['SN','TEST','day','time','datetime','status','TR_PWM_SLOP_FRONT','TR_PWM_OFFSET_FRONT',
              'R2_Front','TR_AccuENC_RATIO_FRONT','TR_PWM_SLOP_REAR','TR_PWM_OFFSET_REAR',
              'R2_Rear','TR_AccuENC_RATIO_REAR']]

In [21]:
#Groupby SN and select last PASS
#https://stackoverflow.com/questions/41525911/group-by-pandas-dataframe-and-select-latest-in-each-group

TR_Pass = TRtest[TRtest.status == 'P'].sort_values('datetime').groupby('SN').tail(1)

#We drop the columns TEST and status because they are no longer useful:
TR_Pass.drop(['TEST', 'status'], axis=1, inplace=True)
TR_Pass.head()

,SN,day,time,datetime,TR_PWM_SLOP_FRONT,TR_PWM_OFFSET_FRONT,R2_Front,TR_AccuENC_RATIO_FRONT,TR_PWM_SLOP_REAR,TR_PWM_OFFSET_REAR,R2_Rear,TR_AccuENC_RATIO_REAR
10990,DRW091810461,20180302,2125,2018030202125,1.370314,-37.246261,0.999998,9.075283,1.372229,-45.169482,0.999979,9.176037
11002,DRW091810471,20180302,4245,2018030204245,1.374144,-28.468581,0.999986,9.075158,1.370026,-40.577860,0.999983,9.174968
561,DRW391710071,20170922,110114,20170922110114,1.335473,-38.879725,0.999897,9.076344,1.377100,-38.177445,0.999997,9.179953
103,DRW131300011,20170926,92330,20170926092330,1.329452,-35.662015,0.999977,9.075938,1.357140,-49.020298,0.999977,9.180915
132,DRW381700121,20170926,102731,20170926102731,1.330637,-36.047478,0.999992,9.075635,1.348320,-40.713569,0.999996,9.179063


In [22]:
TR_Pass.to_csv('../Data/TR.csv', index=False)